<h1>Explore Baseball Data<\h1>

<h3>Ensure that you have set up the baseballDB (see - DBSetUp.ipynb) before running this notebook</h3>

<h4>Import Dependencies</h4>

In [2]:
%matplotlib inline

In [1]:
import os
import csv
import pandas as pd

import sqlite3
import csv
from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import numpy as np

from config import pgPassword

<h4>Create a connection to SQL database</h4>

In [3]:
pg_user = 'postgres'
pg_password = pgPassword
db_name = 'baseball_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

<h4>Read in databases tables as DatFrames</h4>

In [5]:
teamsStatsDF = pd.read_sql_table("Team-Stats", con = engine)
battingDF = pd.read_sql_table("Batting", con = engine)
pitchingDF = pd.read_sql_table("Pitching", con = engine)
playersDF = pd.read_sql_table("Players", con = engine)
franchisesDF = pd.read_sql_table("Franchises", con = engine)
salariesDF = pd.read_sql_table("Salaries", con = engine)
teamsDF = pd.read_sql_table("Teams", con = engine)
fPlayersDF = pd.read_sql_table("FranchisePlayers", con = engine)

In [6]:
battingDF.head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
0,1954,1,35,2,10,0,3,0.0,15.0,0.0,336907024
1,1955,1,46,1,2,0,1,0.0,0.0,0.0,336907024
2,1956,1,49,4,11,0,2,0.0,28.0,0.0,336907024
3,1957,1,49,4,14,0,6,0.0,26.0,0.0,336907024
4,1958,1,46,1,6,0,5,0.0,13.0,1.0,336907024


In [7]:
# Get data going back to latest year of interest
latestDataYear = 1871
targetBattingDF = battingDF[battingDF["yearID"] >=latestDataYear]
targetBattingDF.head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
0,1954,1,35,2,10,0,3,0.0,15.0,0.0,336907024
1,1955,1,46,1,2,0,1,0.0,0.0,0.0,336907024
2,1956,1,49,4,11,0,2,0.0,28.0,0.0,336907024
3,1957,1,49,4,14,0,6,0.0,26.0,0.0,336907024
4,1958,1,46,1,6,0,5,0.0,13.0,1.0,336907024


In [18]:
#normalize the data as perGame stats
targetBattingDF.sort_values(by = ["stint", "fpID"], ascending = False).head()
# targetBattingDF.head(25)

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
261173,2004,4,23,1,8,0,2,0.0,18.0,0.0,695641624
315390,2016,4,14,2,5,2,2,0.0,14.0,0.0,678082599
261172,2004,4,23,1,8,0,2,0.0,18.0,0.0,651373483
315389,2016,4,14,2,5,2,2,0.0,14.0,0.0,634889808
261171,2004,4,23,1,8,0,2,0.0,18.0,0.0,610288615


In [28]:
multipleStints = targetBattingDF.loc[targetBattingDF["stint"] > 1]
multipleStints.shape

(26783, 11)

In [25]:
onePlayer = targetBattingDF.loc[targetBattingDF["fpID"] == 695641624]
onePlayer.sort_values(by = ["yearID"]).head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
261140,2004,1,16,3,3,0,1,0.0,3.0,0.0,695641624
261151,2004,2,12,1,2,0,3,0.0,7.0,0.0,695641624
261162,2004,3,13,1,5,0,1,0.0,12.0,0.0,695641624
261173,2004,4,23,1,8,0,2,0.0,18.0,0.0,695641624
261184,2005,1,11,3,4,0,3,0.0,7.0,1.0,695641624


In [23]:
onePlayer2 = onePlayer.groupby(["yearID", "fpID"]).sum()
onePlayer2.head()

,,stint,G,R,H,HR,BB,IBB,SO,SB
yearID,fpID,,,,,,,,,
2004,695641624,10,64,6,18,0,7,0.0,40.0,0.0
2005,695641624,1,11,3,4,0,3,0.0,7.0,1.0
2006,695641624,1,117,58,94,16,46,2.0,110.0,2.0
2007,695641624,1,142,75,135,15,68,1.0,101.0,6.0
2008,695641624,3,128,45,88,15,40,5.0,91.0,1.0


In [26]:
onePlayer3 = onePlayer2.reset_index()
onePlayer3.head()

,yearID,fpID,stint,G,R,H,HR,BB,IBB,SO,SB
0,2004,695641624,10,64,6,18,0,7,0.0,40.0,0.0
1,2005,695641624,1,11,3,4,0,3,0.0,7.0,1.0
2,2006,695641624,1,117,58,94,16,46,2.0,110.0,2.0
3,2007,695641624,1,142,75,135,15,68,1.0,101.0,6.0
4,2008,695641624,3,128,45,88,15,40,5.0,91.0,1.0


In [ ]:
nums = [83,84, 85,86,87, 88, 89, 90, 91, 92, 95, 96, 97]
lastYear = nums[-1]
gap = 0
gapTimes = 0
for num in nums:
    print(f"===== {num} =====")
    if num >= lastYear:
        continue
    if gapTimes == 2:
        print("Executing 2")
        prev2 = num - 2 - gap
        prev1 = num - 1 - gap
        next1 = num + 1
        gapTimes = 1
    elif gapTimes == 1:
        print("Executing 1")
        prev2 = num - 2 - gap
        prev1 = num - 1
        next1 = num + 1
        gapTimes = 0
    else:
        prev2 = num - 2
        prev1 = num - 1
        next1 = num + 1
    
    while next1 not in nums and next1 <lastYear:
        print(f"Missing {next1}")
        next1 += 1
        gapTimes = 2
        gap = next1 - num - 1
    
    print(prev2, prev1, num, next1)


<h3>=============OLD STUFF=============</h3>

In [39]:
# Merge Players, Batting and Salary Data
mergeDF = pd.merge(targetBattingDF, playersDF, on="playerID")
mergeDF = pd.merge(mergeDF, salariesDF, on= ["yearID", "playerID"])
mergeDF.head()

,playerID,yearID,teamID_x,stint,G,R,H,HR,BB,IBB,SO,SB,birthYear,nameFirst,nameLast,debut,finalGame,teamID_y,salary
0,johnto01,1985,CAL,1,12,0,0,0,0,0.0,0.0,0.0,1943.0,Tommy,John,1963-09-06,1989-05-25,OAK,600000
1,johnto01,1985,OAK,2,11,0,0,0,0,0.0,0.0,0.0,1943.0,Tommy,John,1963-09-06,1989-05-25,OAK,600000
2,johnto01,1986,NYA,1,13,0,0,0,0,0.0,0.0,0.0,1943.0,Tommy,John,1963-09-06,1989-05-25,CAL,440000
3,johnto01,1986,NYA,1,13,0,0,0,0,0.0,0.0,0.0,1943.0,Tommy,John,1963-09-06,1989-05-25,NYA,60000
4,johnto01,1987,NYA,1,33,0,0,0,0,0.0,0.0,0.0,1943.0,Tommy,John,1963-09-06,1989-05-25,NYA,350000


In [33]:
# Normalize statistics 

In [34]:
# Add age and yearsPlayed fileds to table
pbDF["age"] = pbDF.yearID - pbDF.birthYear
pbDF["debutYear"] = pbDF["debut"].dt.year
pbDF["finalYear"] = pbDF["finalGame"].dt.year


In [35]:
pbDF["yearsPlayed"] = pbDF.yearID - pbDF.debutYear
pbDF.head()

,playerID,yearID,teamID,stint,G,R,H,HR,BB,IBB,...,SB,birthYear,nameFirst,nameLast,debut,finalGame,age,debutYear,finalYear,yearsPlayed
0,aaronha01,1970,ATL,1,150,103,154,38,74,15.0,...,9.0,1934.0,Hank,Aaron,1954-04-13,1976-10-03,36.0,1954,1976,16
1,aaronha01,1971,ATL,1,139,95,162,47,71,21.0,...,1.0,1934.0,Hank,Aaron,1954-04-13,1976-10-03,37.0,1954,1976,17
2,aaronha01,1972,ATL,1,129,75,119,34,92,15.0,...,4.0,1934.0,Hank,Aaron,1954-04-13,1976-10-03,38.0,1954,1976,18
3,aaronha01,1973,ATL,1,120,84,118,40,68,13.0,...,1.0,1934.0,Hank,Aaron,1954-04-13,1976-10-03,39.0,1954,1976,19
4,aaronha01,1974,ATL,1,112,47,91,20,39,6.0,...,1.0,1934.0,Hank,Aaron,1954-04-13,1976-10-03,40.0,1954,1976,20
